### Code for the blog **[How to use nested data types effectively in SQL](https://www.startdataengineering.com/post/use-structs-sql/)**

# [SETUP] 

In [2]:
! python ./setup.py

Cleaning up (if any existing) tpch db file tpch.db
Creating TPCH input data at tpch.db


## Connect to DuckDB

In [3]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

There's a new jupysql version available (0.10.14), you're running 0.10.12. To upgrade: pip install jupysql --upgrade
Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [4]:
%%sql
show tables;

Running query in 'duckdb'

name
customer
lineitem
nation
orders
part
partsupp
region
supplier


# [Data Model]

The TPC-H data represents a car parts seller’s data warehouse, where we record orders, items that make up that order (lineitem), supplier, customer, part (parts sold), region, nation, and partsupp (parts supplier). 

Note: Have a copy of the data model as you follow along; this will help in understanding the examples provided and in answering exercise questions.

![](../../tpch_erd.png)


# Using nested data types effectively

##  Use STRUCT for one-to-one & hierarchical relationships

In [5]:
%%sql
-- without nested data type
SELECT l.*,
c.*,
s.*
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

Running query in 'duckdb'

l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment,s_suppkey,s_name,s_address,s_nationkey,s_phone,s_acctbal,s_comment
1,1552,93,1,17.00,24710.35,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull,370,Customer#000000370,DtqbDO5rid,12,22-524-280-8721,8982.79,FURNITURE,nstructions. furious requests across the ironic,93,Supplier#000000093,wNZNHIg370XspE,16,26-528-528-1157,368.76,instructions mold slyly special dolphins. quickly regular instru
1,674,75,2,36.00,56688.12,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui,370,Customer#000000370,DtqbDO5rid,12,22-524-280-8721,8982.79,FURNITURE,nstructions. furious requests across the ironic,75,Supplier#000000075,ULXKdX bZFJwii,18,28-716-704-8686,-224.84,platelets cajole. sentiments
1,637,38,3,8.00,12301.04,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou,370,Customer#000000370,DtqbDO5rid,12,22-524-280-8721,8982.79,FURNITURE,nstructions. furious requests across the ironic,38,Supplier#000000038,"vmOuWWwn,l4j7TOSEw4l3",4,14-361-296-6426,2512.41,inst the slyly final pinto beans. carefully un
1,22,48,4,28.00,25816.56,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t,370,Customer#000000370,DtqbDO5rid,12,22-524-280-8721,8982.79,FURNITURE,nstructions. furious requests across the ironic,48,Supplier#000000048,"IK,6rmdbeaoxtxgx7Df34QfLX5FUcyddqUEbF4h",14,24-722-551-9498,5630.62,ecial excuses detect blithely. slyly regular requests
1,241,23,5,24.00,27389.76,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa",370,Customer#000000370,DtqbDO5rid,12,22-524-280-8721,8982.79,FURNITURE,nstructions. furious requests across the ironic,23,Supplier#000000023,zMyxL7O3O0SUAFoTrY7gVO mnM8XsH,9,19-559-422-5776,5926.41,sual asymptotes affix quickly at the instructions. deposits


![Cols to struct](./images/col-struct.png)

In [6]:
%%sql
-- with nested data types
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

Running query in 'duckdb'

l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment,customer,supplier
1,1552,93,1,17.00,24710.35,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic'}","{'id': 93, 'name': 'Supplier#000000093', 'address': 'wNZNHIg370XspE', 'nationkey': 16, 'phone': '26-528-528-1157', 'acctbal': Decimal('368.76'), 'comment': 'instructions mold slyly special dolphins. quickly regular instru'}"
1,674,75,2,36.00,56688.12,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic'}","{'id': 75, 'name': 'Supplier#000000075', 'address': 'ULXKdX bZFJwii', 'nationkey': 18, 'phone': '28-716-704-8686', 'acctbal': Decimal('-224.84'), 'comment': 'platelets cajole. sentiments '}"
1,637,38,3,8.00,12301.04,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic'}","{'id': 38, 'name': 'Supplier#000000038', 'address': 'vmOuWWwn,l4j7TOSEw4l3', 'nationkey': 4, 'phone': '14-361-296-6426', 'acctbal': Decimal('2512.41'), 'comment': 'inst the slyly final pinto beans. carefully un'}"
1,22,48,4,28.00,25816.56,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic'}","{'id': 48, 'name': 'Supplier#000000048', 'address': 'IK,6rmdbeaoxtxgx7Df34QfLX5FUcyddqUEbF4h', 'nationkey': 14, 'phone': '24-722-551-9498', 'acctbal': Decimal('5630.62'), 'comment': 'ecial excuses detect blithely. slyly regular requests '}"
1,241,23,5,24.00,27389.76,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa","{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic'}","{'id': 23, 'name': 'Supplier#000000023', 'address': ' zMyxL7O3O0SUAFoTrY7gVO mnM8XsH', 'nationkey': 9, 'phone': '19-559-422-5776', 'acctbal': Decimal('5926.41'), 'comment': 'sual asymptotes affix quickly at the instructions. deposits '}"


In [7]:
%%sql
-- Hierarchical data 
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment,
        nation := struct_pack(
            nationkey := sn.n_nationkey,
            name := sn.n_name,
            regionkey := sn.n_regionkey,
            comment := sn.n_comment
            )
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LEFT JOIN
    nation sn ON s.s_nationkey = sn.n_nationkey
LIMIT 5;

Running query in 'duckdb'

l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment,customer,supplier
1,1552,93,1,17.00,24710.35,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic', 'nation': {'nationkey': 12, 'name': 'JAPAN', 'regionkey': 2, 'comment': ' quickly final packages. furiously i'}}","{'id': 93, 'name': 'Supplier#000000093', 'address': 'wNZNHIg370XspE', 'nationkey': 16, 'phone': '26-528-528-1157', 'acctbal': Decimal('368.76'), 'comment': 'instructions mold slyly special dolphins. quickly regular instru', 'nation': {'nationkey': 16, 'name': 'MOZAMBIQUE', 'regionkey': 0, 'comment': ' beans after the carefully regular accounts r'}}"
1,674,75,2,36.00,56688.12,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic', 'nation': {'nationkey': 12, 'name': 'JAPAN', 'regionkey': 2, 'comment': ' quickly final packages. furiously i'}}","{'id': 75, 'name': 'Supplier#000000075', 'address': 'ULXKdX bZFJwii', 'nationkey': 18, 'phone': '28-716-704-8686', 'acctbal': Decimal('-224.84'), 'comment': 'platelets cajole. sentiments ', 'nation': {'nationkey': 18, 'name': 'CHINA', 'regionkey': 2, 'comment': 'ckly special packages cajole slyly. unusual, unusual theodolites mold furiously. slyly sile'}}"
1,637,38,3,8.00,12301.04,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic', 'nation': {'nationkey': 12, 'name': 'JAPAN', 'regionkey': 2, 'comment': ' quickly final packages. furiously i'}}","{'id': 38, 'name': 'Supplier#000000038', 'address': 'vmOuWWwn,l4j7TOSEw4l3', 'nationkey': 4, 'phone': '14-361-296-6426', 'acctbal': Decimal('2512.41'), 'comment': 'inst the slyly final pinto beans. carefully un', 'nation': {'nationkey': 4, 'name': 'EGYPT', 'regionkey': 4, 'comment': 'usly ironic, pending foxes. even, special instructions nag. sly, final foxes detect slyly fluffily '}}"
1,22,48,4,28.00,25816.56,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t,"{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic', 'nation': {'nationkey': 12, 'name': 'JAPAN', 'regionkey': 2, 'comment': ' quickly final packages. furiously i'}}","{'id': 48, 'name': 'Supplier#000000048', 'address': 'IK,6rmdbeaoxtxgx7Df34QfLX5FUcyddqUEbF4h', 'nationkey': 14, 'phone': '24-722-551-9498', 'acctbal': Decimal('5630.62'), 'comment': 'ecial excuses detect blithely. slyly regular requests ', 'nation': {'nationkey': 14, 'name': 'KENYA', 'regionkey': 0, 'comment': 'lyly special foxes. slyly regular deposits sleep carefully. carefully permanent accounts slee'}}"
1,241,23,5,24.00,27389.76,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa","{'id': 370, 'name': 'Customer#000000370', 'address': 'DtqbDO5rid', 'nationkey': 12, 'phone': '22-524-280-8721', 'acctbal': Decimal('8982.79'), 'mktsegment': 'FURNITURE', 'comment': 'nstructions. furious requests across the ironic', 'nation': {'nationkey': 12, 'name': 'JAPAN', 'regionkey': 2, 'comme

In [8]:
# exercise, to the above query add region attributes

## Use ARRAY[STRUCT] for one-to-many relationships

![Rows to List](./images/row-list.png)

In [9]:
%%sql
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT o.*,
len(l.lineitems) as num_lineitems,
l.lineitems
FROM orders o
LEFT JOIN line_items l
on o.o_orderkey = l.orderkey
LIMIT 5;

Running query in 'duckdb'

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment,num_lineitems,lineitems
1,370,O,172799.49,1996-01-02,5-LOW,Clerk#000000951,0,ly express platelets. deposits acc,6,"[{'lineitemkey': 1, 'partkey': 1552, 'suppkey': 93, 'quantity': Decimal('17.00'), 'extendedprice': Decimal('24710.35'), 'discount': Decimal('0.04'), 'tax': Decimal('0.02'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 3, 13), 'commitdate': datetime.date(1996, 2, 12), 'receiptdate': datetime.date(1996, 3, 22), 'shipinstruct': 'DELIVER IN PERSON', 'shipmode': 'TRUCK', 'comment': 'to beans x-ray carefull'}, {'lineitemkey': 2, 'partkey': 674, 'suppkey': 75, 'quantity': Decimal('36.00'), 'extendedprice': Decimal('56688.12'), 'discount': Decimal('0.09'), 'tax': Decimal('0.06'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 4, 12), 'commitdate': datetime.date(1996, 2, 28), 'receiptdate': datetime.date(1996, 4, 20), 'shipinstruct': 'TAKE BACK RETURN', 'shipmode': 'MAIL', 'comment': ' according to the final foxes. qui'}, {'lineitemkey': 3, 'partkey': 637, 'suppkey': 38, 'quantity': Decimal('8.00'), 'extendedprice': Decimal('12301.04'), 'discount': Decimal('0.10'), 'tax': Decimal('0.02'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 1, 29), 'commitdate': datetime.date(1996, 3, 5), 'receiptdate': datetime.date(1996, 1, 31), 'shipinstruct': 'TAKE BACK RETURN', 'shipmode': 'REG AIR', 'comment': 'ourts cajole above the furiou'}, {'lineitemkey': 4, 'partkey': 22, 'suppkey': 48, 'quantity': Decimal('28.00'), 'extendedprice': Decimal('25816.56'), 'discount': Decimal('0.09'), 'tax': Decimal('0.06'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 4, 21), 'commitdate': datetime.date(1996, 3, 30), 'receiptdate': datetime.date(1996, 5, 16), 'shipinstruct': 'NONE', 'shipmode': 'AIR', 'comment': 's cajole busily above t'}, {'lineitemkey': 5, 'partkey': 241, 'suppkey': 23, 'quantity': Decimal('24.00'), 'extendedprice': Decimal('27389.76'), 'discount': Decimal('0.10'), 'tax': Decimal('0.04'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 3, 30), 'commitdate': datetime.date(1996, 3, 14), 'receiptdate': datetime.date(1996, 4, 1), 'shipinstruct': 'NONE', 'shipmode': 'FOB', 'comment': ' the regular, regular pa'}, {'lineitemkey': 6, 'partkey': 157, 'suppkey': 10, 'quantity': Decimal('32.00'), 'extendedprice': Decimal('33828.80'), 'discount': Decimal('0.07'), 'tax': Decimal('0.02'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 1, 30), 'commitdate': datetime.date(1996, 2, 7), 'receiptdate': datetime.date(1996, 2, 3), 'shipinstruct': 'DELIVER IN PERSON', 'shipmode': 'MAIL', 'comment': 'rouches. special '}]"
2,781,O,38426.09,1996-12-01,1-URGENT,Clerk#000000880,0,ve the furiously fluffy dependencies. carefully regular,1,"[{'lineitemkey': 1, 'partkey': 1062, 'suppkey': 33, 'quantity': Decimal('38.00'), 'extendedprice': Decimal('36596.28'), 'discount': Decimal('0.00'), 'tax': Decimal('0.05'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1997, 1, 28), 'commitdate': datetime.date(1997, 1, 14), 'receiptdate': datetime.date(1997, 2, 2), 'shipinstruct': 'TAKE BACK RETURN', 'shipmode': 'RAIL', 'comment': 're. enticingly regular instruct'}]"
3,1234,F,205654.30,1993-10-14,5-LOW,Clerk#000000955,0,after the asymptotes. instructions cajole after the foxes. carefully unu,6,"[{'lineitemkey': 1, 'partkey': 43, 'suppkey': 19, 'quantity': Decimal('45.00'), 'extendedprice': Decimal('42436.80'), 'discount': Decimal('0.06'), 'tax': Decimal('0.00'), 'returnflag': 'R', 'linestatus': 'F', 'shipdate': datetime.date(1994, 2, 2), 'commitdate': datetime.date(1994, 1, 4), 'receiptdate': datetime.date(1994, 2, 23), 'shipinstruct': 'NONE', 'shipmode': 'AIR', 'comment': 's cajole above the pinto beans. iro'}, {'lineitemkey': 2, 'partkey': 191, 'suppkey': 70, 'quantity': Decimal('49.00'), 'extendedprice': Decimal

## Using nested data types in data processing

In [10]:
%%sql
DROP TABLE IF EXISTS wide_orders;

Running query in 'duckdb'

Success


In [11]:
%%sql
CREATE TABLE IF NOT EXISTS wide_orders AS 
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT 
    o.*,
    l.lineitems,
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer
FROM 
    orders o
LEFT JOIN 
    line_items l ON o.o_orderkey = l.orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey;

Running query in 'duckdb'

Count
15000


### STRUCT enables more straightforward data schema and data access

In [12]:
%%sql
SELECT o_orderkey,
customer.name,
customer.address,
lineitems[1] as first_lineitem
FROM wide_orders
limit 2;

Running query in 'duckdb'

o_orderkey,name,address,first_lineitem
1,Customer#000000370,DtqbDO5rid,"{'lineitemkey': 1, 'partkey': 1552, 'suppkey': 93, 'quantity': Decimal('17.00'), 'extendedprice': Decimal('24710.35'), 'discount': Decimal('0.04'), 'tax': Decimal('0.02'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1996, 3, 13), 'commitdate': datetime.date(1996, 2, 12), 'receiptdate': datetime.date(1996, 3, 22), 'shipinstruct': 'DELIVER IN PERSON', 'shipmode': 'TRUCK', 'comment': 'to beans x-ray carefull'}"
2,Customer#000000781,"maoqGuL5,rHfX0leqZcFqHqpQH","{'lineitemkey': 1, 'partkey': 1062, 'suppkey': 33, 'quantity': Decimal('38.00'), 'extendedprice': Decimal('36596.28'), 'discount': Decimal('0.00'), 'tax': Decimal('0.05'), 'returnflag': 'N', 'linestatus': 'O', 'shipdate': datetime.date(1997, 1, 28), 'commitdate': datetime.date(1997, 1, 14), 'receiptdate': datetime.date(1997, 2, 2), 'shipinstruct': 'TAKE BACK RETURN', 'shipmode': 'RAIL', 'comment': 're. enticingly regular instruct'}"


### Nested data types can be sorted



In [13]:
%%sql
SELECT customer
FROM wide_orders 
GROUP BY 1
ORDER BY 1
LIMIT 5


Running query in 'duckdb'

customer
"{'id': 1, 'name': 'Customer#000000001', 'address': 'j5JsirBM9PsCy0O1m', 'nationkey': 15, 'phone': '25-989-741-2988', 'acctbal': Decimal('711.56'), 'mktsegment': 'BUILDING', 'comment': 'y final requests wake slyly quickly special accounts. blithely', 'nation': {'nationkey': 15, 'name': 'MOROCCO', 'regionkey': 0, 'comment': 'ct blithely: blithely express accounts nag carefully. silent packages haggle carefully abo'}}"
"{'id': 2, 'name': 'Customer#000000002', 'address': '487LW1dovn6Q4dMVymKwwLE9OKf3QG', 'nationkey': 13, 'phone': '23-768-687-3665', 'acctbal': Decimal('121.65'), 'mktsegment': 'AUTOMOBILE', 'comment': 'y carefully regular foxes. slyly regular requests about the bli', 'nation': {'nationkey': 13, 'name': 'JORDAN', 'regionkey': 4, 'comment': 'the slyly regular ideas. silent Tiresias affix slyly fu'}}"
"{'id': 4, 'name': 'Customer#000000004', 'address': '4u58h fqkyE', 'nationkey': 4, 'phone': '14-128-190-5944', 'acctbal': Decimal('2866.83'), 'mktsegment': 'MACHINERY', 'comment': ' sublate. fluffily even instructions are about th', 'nation': {'nationkey': 4, 'name': 'EGYPT', 'regionkey': 4, 'comment': 'usly ironic, pending foxes. even, special instructions nag. sly, final foxes detect slyly fluffily '}}"
"{'id': 5, 'name': 'Customer#000000005', 'address': 'hwBtxkoBF qSW4KrIk5U 2B1AU7H', 'nationkey': 3, 'phone': '13-750-942-6364', 'acctbal': Decimal('794.47'), 'mktsegment': 'HOUSEHOLD', 'comment': 'equests haggle furiously against the pending packa', 'nation': {'nationkey': 3, 'name': 'CANADA', 'regionkey': 1, 'comment': 'ss deposits wake across the pending foxes. packages after the carefully bold requests integrate caref'}}"
"{'id': 7, 'name': 'Customer#000000007', 'address': '8OkMVLQ1dK6Mbu6WG9 w4pLGQ n7MQ', 'nationkey': 18, 'phone': '28-190-982-9759', 'acctbal': Decimal('9561.95'), 'mktsegment': 'AUTOMOBILE', 'comment': 'ounts. ironic, regular accounts sleep. final requests haggle quickly after the', 'nation': {'nationkey': 18, 'name': 'CHINA', 'regionkey': 2, 'comment': 'ckly special packages cajole slyly. unusual, unusual theodolites mold furiously. slyly sile'}}"


In [14]:
# exercise: Sort ARRAY of lineitems, how is it sorted? What do you think is the sort order based on?

### UNNEST ARRAY to rows and GROUP rows to ARRAY

In [15]:
%%sql
-- rows to ARRAY
WITH lineitems as (SELECT 
    o.o_orderkey,
    UNNEST(o.lineitems) as line_item
FROM 
    wide_orders o),
unnested_line_items AS (
SELECT o_orderkey,
    line_item.lineitemkey,
    line_item.partkey,
    line_item.quantity
    FROM lineitems
)
SELECT o_orderkey,
array_agg(struct_pack(
        line_item_key := lineitemkey,
        part_key := partkey,
        quantity := quantity)) as lineitems
FROM unnested_line_items
GROUP BY 1
LIMIT 5;

Running query in 'duckdb'

o_orderkey,lineitems
1,"[{'line_item_key': 1, 'part_key': 1552, 'quantity': Decimal('17.00')}, {'line_item_key': 2, 'part_key': 674, 'quantity': Decimal('36.00')}, {'line_item_key': 3, 'part_key': 637, 'quantity': Decimal('8.00')}, {'line_item_key': 4, 'part_key': 22, 'quantity': Decimal('28.00')}, {'line_item_key': 5, 'part_key': 241, 'quantity': Decimal('24.00')}, {'line_item_key': 6, 'part_key': 157, 'quantity': Decimal('32.00')}]"
33,"[{'line_item_key': 1, 'part_key': 614, 'quantity': Decimal('31.00')}, {'line_item_key': 2, 'part_key': 606, 'quantity': Decimal('32.00')}, {'line_item_key': 3, 'part_key': 1375, 'quantity': Decimal('5.00')}, {'line_item_key': 4, 'part_key': 340, 'quantity': Decimal('41.00')}]"
68,"[{'line_item_key': 1, 'part_key': 71, 'quantity': Decimal('3.00')}, {'line_item_key': 2, 'part_key': 1752, 'quantity': Decimal('46.00')}, {'line_item_key': 3, 'part_key': 350, 'quantity': Decimal('46.00')}, {'line_item_key': 4, 'part_key': 948, 'quantity': Decimal('20.00')}, {'line_item_key': 5, 'part_key': 828, 'quantity': Decimal('27.00')}, {'line_item_key': 6, 'part_key': 1026, 'quantity': Decimal('30.00')}, {'line_item_key': 7, 'part_key': 1393, 'quantity': Decimal('41.00')}]"
98,"[{'line_item_key': 1, 'part_key': 403, 'quantity': Decimal('28.00')}, {'line_item_key': 2, 'part_key': 1098, 'quantity': Decimal('1.00')}, {'line_item_key': 3, 'part_key': 448, 'quantity': Decimal('14.00')}, {'line_item_key': 4, 'part_key': 1672, 'quantity': Decimal('10.00')}]"
133,"[{'line_item_key': 1, 'part_key': 1035, 'quantity': Decimal('27.00')}, {'line_item_key': 2, 'part_key': 1763, 'quantity': Decimal('12.00')}, {'line_item_key': 3, 'part_key': 1174, 'quantity': Decimal('29.00')}, {'line_item_key': 4, 'part_key': 899, 'quantity': Decimal('11.00')}]"


In [16]:
%%sql
-- ARRAY to rows
WITH
  lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
  )
SELECT
  o_orderkey,
  line_item.lineitemkey,
  line_item.partkey,
  line_item.quantity
FROM
  lineitems
LIMIT
  5


Running query in 'duckdb'

o_orderkey,lineitemkey,partkey,quantity
1,1,1552,17.00
1,2,674,36.00
1,3,637,8.00
1,4,22,28.00
1,5,241,24.00


### Improve OBT usability with nested data types

![Nested OBT to pre aggregate](./images/nested-preagg.png)

In [22]:
%%sql
-- get lineitem metrics, with array aggregation functions (specific to DuckDB)
SELECT o_orderkey,
    o_totalprice AS order_total_price,
    len(lineitems) as num_line_items,
    array_reduce(array_transform(lineitems, x -> x.quantity), (x, y) -> x + y) as total_line_item_quantity
FROM wide_orders
ORDER BY
  1
LIMIT
  10;


Running query in 'duckdb'

o_orderkey,order_total_price,num_line_items,total_line_item_quantity
1,172799.49,6,145.00
2,38426.09,1,38.00
3,205654.30,6,177.00
4,56000.91,1,30.00
5,105367.67,3,91.00
6,45523.10,1,37.00
7,271885.66,7,173.00
32,198665.57,6,116.00
33,146567.24,4,109.00
34,73315.48,3,41.00


In [18]:
%%sql
-- Get order metrics
SELECT
  o_orderdate,
  SUM(o_totalprice) AS order_total_price
FROM
  wide_orders
GROUP BY
  1
ORDER BY
  1
LIMIT
  5;

Running query in 'duckdb'

o_orderdate,order_total_price
1992-01-01,1359507.29
1992-01-02,916463.63
1992-01-03,1020749.95
1992-01-04,1503029.23
1992-01-05,1124149.60


In [ ]:
%%sql
-- get lineitem metrics
WITH
  lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
  )
SELECT
  o_orderkey,
  COUNT(line_item.lineitemkey) AS num_line_items,
  SUM(line_item.quantity) AS total_line_item_quantity
FROM
  lineitems
GROUP BY
  1
ORDER BY
  1
LIMIT
  10;


In [ ]:
# exercise: create a wide_orders_v2 table that combines all the tables in the TPCH data model

## Ensure your performance meets your expectations

In [ ]:
result = %sql EXPLAIN WITH lineitems AS ( SELECT o.o_orderkey, UNNEST (o.lineitems) AS line_item FROM wide_orders o ) SELECT o_orderkey, COUNT(line_item.lineitemkey) AS num_line_items, SUM(line_item.quantity) AS total_line_item_quantity FROM lineitems GROUP BY 1 ORDER BY 1
print(result)